<a href="https://colab.research.google.com/github/tetsufmbio/remolog/blob/main/notebooks/remolog_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remolog: identifying remote homologs using structural alignment data

## Quick start

1. Press "Runtime" --> "Run all".
2. In the next cell (*Upload protein structure*), it will appear the bottom "Choose file". Click on it and choose one or more pdb file to be uploaded and analyzed by the pipeline*.
3. After the running, it will download a file named "remolog_final_result.tab". See its description in the end of this notebook.
4. Running time of this pipeline in Colab take ~10 min when using foldseek for screening**.

\* If you have only the amino acid sequence, you can predict its structure using the [AlphaFold Colab notebook](https://github.com/sokrypton/ColabFold)

\** If no remote homolog was found, try to increase the parameter "n" or change the screening method.


In [1]:
%cd /content
! if [ -d input ]; then \
    rm -rf input; \
  fi
! mkdir input


%cd input

from google.colab import files

uploaded = files.upload()
#@markdown  By running this cell, a bottom "Choose files" may appear. Click on it and choose the pdb file to be analyzed (you may upload multiple files).

%cd /content


/content
/content/input


Saving WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb to WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb
/content


## Setting parameters

In [2]:
screening = "foldseek" #@param ["foldseek","tmalign", "fatcat"]
#@markdown  - software to be used to retrieve structurely similar proteins in SCOPe database.

n = 20 #@param {type:"integer"}
#@markdown  - analyze n most similar protein structures.

database = "scope40" #@param ["scope40", "scope95"]
#@markdown  - protein structure database to be used.

jobName = "remolog_final_result" #@param {type: "string"}
#@markdown  - prefix for the final output name.


## Installing dependencies and downloading database

In [3]:
# setup environment variables
import os
os.environ['FATCAT'] = '/content/programs/FATCAT-dist'
os.environ['PATH'] += ':/content/programs/FATCAT-dist/FATCATMain:/content/bin'
os.environ['HEADN'] = str(n)
os.environ['SCREEN'] = screening

In [4]:
%cd /content/
! if [ ! -d bin ]; then mkdir bin; fi
! if [ ! -d programs ]; then mkdir programs; fi
! if [ ! -d view ]; then mkdir view; fi

/content


In [5]:
# download and install TMalign
%cd /content/bin
! if [ ! -e TMalign ]; then \
    wget "https://zhanggroup.org/TM-align/TMalign.cpp"; \
    g++ -static -O3 -ffast-math -lm -o TMalign TMalign.cpp; \
  fi


/content/bin
--2023-04-17 21:15:04--  https://zhanggroup.org/TM-align/TMalign.cpp
Resolving zhanggroup.org (zhanggroup.org)... 141.213.137.249
Connecting to zhanggroup.org (zhanggroup.org)|141.213.137.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182097 (178K) [text/plain]
Saving to: ‘TMalign.cpp’

TMalign.cpp         100%[===================>] 177.83K  --.-KB/s    in 0.09s   

2023-04-17 21:15:05 (1.85 MB/s) - ‘TMalign.cpp’ saved [182097/182097]



In [6]:
# download and install FATCAT
%cd /content/programs
! if [ ! -d FATCAT-dist ]; then \
    git clone https://github.com/GodzikLab/FATCAT-dist.git; \
    cd FATCAT-dist/; ./Install; \
  fi


/content/programs
Cloning into 'FATCAT-dist'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 119 (delta 17), reused 109 (delta 13), pack-reused 0
Receiving objects: 100% (119/119), 2.37 MiB | 15.20 MiB/s, done.
Resolving deltas: 100% (17/17), done.
g++ -O2 -Wall -I./ -c SAlnOpt.C
g++ -O2 -Wall -I./ -c Prot.C
g++ -O2 -Wall -I./ -c AFPchain.C
AFPchain.C:872:1: warning: multi-line comment [-Wcomment]
  872 | //       j1 -|---------------------\
      | ^
AFPchain.C: In member function ‘void AFPCHAIN::MergeAfp(int, int*, int*)’:
AFPchain.C:445:35: warning: variable ‘j2’ set but not used [-Wunused-but-set-variable]
  445 |  int i, j, k, f1, f2, i1, i2, j1, j2, frag;
      |                                   ^~
AFPchain.C: In member function ‘void AFPCHAIN::UpdateScore()’:
AFPchain.C:1154:9: warning: variable ‘t’ set but not used [-Wunused-but-set-variable]
 1154 |  double t, conn, dva

In [7]:
# download and install lovoalign
%cd /content/bin
! if [ ! -e lovoalign ]; then \
    wget "https://github.com/m3g/lovoalign/archive/refs/tags/22.0.0.tar.gz"; \
    tar -xzf 22.0.0.tar.gz; cd lovoalign-22.0.0/src; make; cp ../bin/lovoalign /content/bin; \
  fi

/content/bin
--2023-04-17 21:15:21--  https://github.com/m3g/lovoalign/archive/refs/tags/22.0.0.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/m3g/lovoalign/tar.gz/refs/tags/22.0.0 [following]
--2023-04-17 21:15:21--  https://codeload.github.com/m3g/lovoalign/tar.gz/refs/tags/22.0.0
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘22.0.0.tar.gz’

22.0.0.tar.gz           [ <=>                ]  31.57K  --.-KB/s    in 0.01s   

2023-04-17 21:15:21 (2.34 MB/s) - ‘22.0.0.tar.gz’ saved [32325]

 ------------------------------------------------------ 
 Compiling LovoAlign with gfortran 
 Flags: -O3 -ffast-math -llapack  
 -

In [8]:
# download some scripts and model
%cd /content/programs
! if [ ! -d remolog ]; then \
    git clone https://github.com/tetsufmbio/remolog.git; \
  fi

/content/programs
Cloning into 'remolog'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 140 (delta 63), reused 86 (delta 23), pack-reused 0
Receiving objects: 100% (140/140), 16.52 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [9]:
%cd /content/programs
! if [ ! -e /content/bin/foldseek ]; then \
    wget https://mmseqs.com/foldseek/foldseek-linux-sse2.tar.gz; tar xvzf foldseek-linux-sse2.tar.gz; \
    cp foldseek/bin/foldseek /content/bin; \
  fi

/content/programs
--2023-04-17 21:15:30--  https://mmseqs.com/foldseek/foldseek-linux-sse2.tar.gz
Resolving mmseqs.com (mmseqs.com)... 141.5.100.26
Connecting to mmseqs.com (mmseqs.com)|141.5.100.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41044169 (39M) [application/octet-stream]
Saving to: ‘foldseek-linux-sse2.tar.gz’

foldseek-linux-sse2 100%[===================>]  39.14M  19.5MB/s    in 2.0s    

2023-04-17 21:15:32 (19.5 MB/s) - ‘foldseek-linux-sse2.tar.gz’ saved [41044169/41044169]

foldseek/
foldseek/README.md
foldseek/bin/
foldseek/bin/foldseek


In [10]:
# download and format scope database
%cd /content
! if [ ! -d database ]; then \
    mkdir database; \
  fi;
%cd /content/database
! if [[ $database = "scope40" ]]; then \
    wget https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-sel-gs-bib-40-2.08.tgz; \
    tar -zxf pdbstyle-sel-gs-bib-40-2.08.tgz; mv pdbstyle-2.08/*/*.ent . ; rm -rf pdbstyle*; \
  elif [[ $database = "scope95" ]]; then \
    wget https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-sel-gs-bib-95-2.08.tgz; \
    tar -zxf pdbstyle-sel-gs-bib-95-2.08.tgz; mv pdbstyle-2.08/*/*.ent . ; rm -rf pdbstyle*; \
  fi; 
! ls *.ent > ../list_scope.tab; for i in *.ent; do mv $i $i.pdb; done;


/content
/content/database
--2023-04-17 21:15:35--  https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-sel-gs-bib-40-2.08.tgz
Resolving scop.berkeley.edu (scop.berkeley.edu)... 128.32.236.13
Connecting to scop.berkeley.edu (scop.berkeley.edu)|128.32.236.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1040288513 (992M) [application/x-gzip]
Saving to: ‘pdbstyle-sel-gs-bib-40-2.08.tgz’

pdbstyle-sel-gs-bib 100%[===================>] 992.10M  24.8MB/s    in 34s     

2023-04-17 21:16:09 (29.6 MB/s) - ‘pdbstyle-sel-gs-bib-40-2.08.tgz’ saved [1040288513/1040288513]



In [11]:
# creating foldseek database
%cd /content
! if [ ! -d foldseek_data ]; then \
  mkdir foldseek_data; cd /content/foldseek_data; \
  foldseek createdb /content/database/ fs_data ; \
fi


/content
createdb /content/database/ fs_data 

MMseqs Version:        	543191f6dc9674607fdd6c0ecc173d9d748150ac
Chain name mode        	0
Mask b-factor threshold	0
Coord store mode       	2
Write lookup file      	1
Tar Inclusion Regex    	.*
Tar Exclusion Regex    	^$
Threads                	2
Verbosity              	3

Output file: fs_data
[=================================================================] 100.00% 15.18K 43s 813ms
Time for merging to fs_data_ss: 0h 0m 0s 20ms
Time for merging to fs_data_h: 0h 0m 0s 11ms
Time for merging to fs_data_ca: 0h 0m 0s 55ms
Time for merging to fs_data: 0h 0m 0s 21ms
Ignore 0 out of 39611.
Too short: 0, incorrect  0.
Time for processing: 0h 0m 44s 87ms


## Running protein structure alignment

In [12]:
%cd /content
! if [ -d result ]; then rm -rf result ; fi
! mkdir result;
! mkdir result/screening;

/content


In [13]:
# screening for similar proteins using foldseek
%cd /content/input

! if [ $SCREEN = "foldseek" ]; then \
    for f in *; do \
      foldseek easy-search $f /content/foldseek_data/fs_data /content/result/screening/tmp.tab.fmt /content/tmpFolder --max-seqs $HEADN -e inf; \
      cut -f 1,2 /content/result/screening/tmp.tab.fmt | sort | uniq | perl -ne '@a = split(/\./, $_); print join(".", @a[0 .. $#a-2])."\n";' > /content/result/screening/$f.tab.fmt; \
      rm /content/result/screening/tmp.tab.fmt; \
    done; \
  fi

/content/input
Create directory /content/tmpFolder
easy-search WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb /content/foldseek_data/fs_data /content/result/screening/tmp.tab.fmt /content/tmpFolder --max-seqs 20 -e inf 

MMseqs Version:              	543191f6dc9674607fdd6c0ecc173d9d748150ac
Seq. id. threshold           	0
Coverage threshold           	0
Coverage mode                	0
Max reject                   	2147483647
Max accept                   	2147483647
Add backtrace                	false
TMscore threshold            	0
TMalign hit order            	0
TMalign fast                 	1
Preload mode                 	0
Threads                      	2
Verbosity                    	3
LDDT threshold               	0
Sort by structure bit score  	1
Substitution matrix          	aa:3di.out,nucl:3di.out
Alignment mode               	3
Alignment mode               	0
E-value threshold            	inf
Min alignment length         	0
Seq. id. mode             

In [14]:
# screening for similar proteins using FATCAT

%cd /content/input
! if [ $SCREEN = "fatcat" ]; then \
    for f in *; do \
      FATCATSearch.pl $f /content/list_scope.tab -b -i1 /content/input -i2 /content/database | \
      sort -k11nr | \
      head -n $HEADN | \
      perl /content/programs/remolog/scripts/format_result_FATCAT.pl - /content/programs/remolog/data/maxScore_fatcat.tab > /content/result/screening/$f.tab.fmt; \
    done; \
    cat /content/result/screening/*.fmt > /content/result/fatcat_formatted.tab; \
  fi

/content/input


In [15]:
# screening for similar proteins using tmalign

%cd /content/input
! if [ $SCREEN = "tmalign" ]; then \
  for f in *; do \
    if [ -f /content/result/screening/${f}.tab ]; then \
      rm /content/result/screening/${f}.tab; \
    fi; \
    for l in $(cat /content/list_scope.tab); \
      do TMalign /content/input/$f /content/database/${l}.pdb | perl /content/programs/remolog/scripts/parser_TMalign.pl - >> /content/result/screening/${f}.tab ; \
      done; \
    sort -k3nr /content/result/screening/${f}.tab | grep ${f} | head -n $HEADN > /content/result/screening/${f}.tab.fmt; \
    done; \
  cat /content/result/screening/*.fmt > /content/result/tmalign_formatted.tab; \
fi

/content/input


In [16]:
%cd /content/input
! if [ $SCREEN != "fatcat" ]; then \
  if [ -f /content/result/fatcat_formatted.tab ]; then \
    rm /content/result/fatcat_formatted.tab; \
  fi; \
  for f in *; do \
    for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
      FATCAT -p1 $f -p2 $l.ent.pdb -i1 /content/input -i2 /content/database -b | \
      perl /content/programs/remolog/scripts/format_result_FATCAT.pl - /content/programs/remolog/data/maxScore_fatcat.tab >>  /content/result/fatcat_formatted.tab ; \
    done; \
  done; \
  fi

/content/input


In [17]:
%cd /content/input
! if [ $SCREEN != "tmalign" ]; then \
    if [ -f /content/result/tmalign_formatted.tab ]; then \
      rm /content/result/tmalign_formatted.tab; \
    fi; \
    for f in *; do \
      for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
        TMalign /content/input/$f /content/database/$l.ent.pdb | perl /content/programs/remolog/scripts/parser_TMalign.pl - >> /content/result/tmalign_formatted.tab ; \
      done; \
    done;\
  fi

/content/input


In [18]:
%cd /content/input
! if [ $SCREEN != "lovoalign" ]; then \
    if [ -f /content/result/lovoalign_formatted.tab ]; then \
      rm /content/result/lovoalign_formatted.tab; \
    fi; \
    for f in *; do \
      for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
        lovoalign -p1 /content/input/$f -p2 /content/database/$l.ent.pdb | perl /content/programs/remolog/scripts/parser_lovoalign.pl - >> /content/result/lovoalign_formatted.tab; \
      done; \
    done; \
  fi

/content/input


In [19]:
%cd /content/result
! perl /content/programs/remolog/scripts/join_table.pl /content/result/fatcat_formatted.tab /content/result/tmalign_formatted.tab | \
  perl /content/programs/remolog/scripts/join_table.pl - /content/result/lovoalign_formatted.tab | \
  perl /content/programs/remolog/scripts/add_scope_class.pl - /content/programs/remolog/data/dir.cla.scope.2.08-stable_filtered40.txt > result.tab

/content/result


## Making prediction and writing the results

In [20]:
from joblib import load
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

clf = load('/content/programs/remolog/model.joblib')

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RobustScaler from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refe

In [21]:
header = ["query", "subject",
          "lovo_finalScore", "lovo_coverage", "lovo_rmsd", "lovo_gaps", "lovo_relCov", "lovo_relGaps", "lovo_finalScoreNorm",
          "tm_AliLen", "tm_RMSD", "tm_n_ident/n_aln", "tm_TM-score (chain 2)", "tm_d0 (chain 2)","tm_cov",
          "fatcat_subject-len", "fatcat_Twists", "fatcat_ini-len", "fatcat_ini-rmsd", "fatcat_opt-equ", "fatcat_opt-rmsd", "fatcat_chain-rmsd", "fatcat_Score", "fatcat_align-len", "fatcat_Gaps", "fatcat_rel_score", "fatcat_rel_align",
          'cl=46456', 'cl=48724', 'cl=51349', 'cl=53931', 'cl=56572', 'cl=56835', 'cl=56992',
            ]
data = pd.read_csv("/content/result/result.tab", sep="\t", header=None)
data.columns = header

annot = pd.read_csv("/content/programs/remolog/data/dir.cla.scope.2.08-stable_filtered40.txt", header=None, sep="\t")
annot[['cl','cf','sf','fa','dm','sp','px']] = annot[5].str.split(',',expand=True)


In [22]:
X = data.iloc[:,2:]
pred = clf.predict(X)
pred_proba = clf.predict_proba(X)
data["pred"] = pred
data["pred_proba"] = pred_proba[:,1]

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


In [23]:
annot.index = annot.loc[:, 0]
annot = annot.loc[:,["cl","cf","sf"]]
data = data.join(annot, on="subject")

In [24]:
data = data.sort_values(by=["query", "pred_proba"], ascending=[True, False])

In [25]:
# write and download result file
from google.colab import files

data.to_csv("/content/result/"+jobName+".tab", index=False)
files.download("/content/result/"+jobName+".tab")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
# function to display table
from google.colab import data_table
data_table.enable_dataframe_formatter()

def hyperlink(path):
	
    # returns the substring of a path

    pathList = path.split("=")
    f_url = pathList[len(pathList)-1]
    path="https://scop.berkeley.edu/search/?ver=2.08&key="+f_url
    #print(f_url)
    
    # convert the path into clickable link
    return '<a target="_blank" href="{}">{}</a>'.format(path, f_url)


In [27]:
# functions to display 3d alignment
! pip install py3Dmol
import py3Dmol
import glob
import matplotlib.pyplot as plt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import ipywidgets as widgets

def print_query(subject):
    print(subject)

def select_subject(query):
    subject_picker.options = list(data[data["query"] == query].subject)

query_list = data.loc[:,"query"].unique()
query_picker = widgets.Dropdown(options=query_list, value=query_list[0])
subject_list = list(data[data["query"] == query_list[0]].subject)
subject_picker = widgets.Dropdown(options=subject_list, value=subject_list[0])
j = widgets.interactive(print_query, subject=subject_picker)
i = widgets.interactive(select_subject, query=query_picker)

#button = widgets.Button(description="Submit")
#output = widgets.Output()
#display(button)

def display_str():
  selected_subject = data[(data["subject"] == subject_picker.value) & (data["query"] == query_picker.value)]
  os.chdir("/content/view")
  os.system("rm *")
  os.system("FATCAT -p1 "+query_picker.value+" -p2 "+subject_picker.value+".ent.pdb -i1 /content/input -i2 /content/database -t")

  with open("/content/view/tmp.opt.twist.pdb") as ifile:
      system = "".join([x for x in ifile])
      
  view = py3Dmol.view(width=400, height=300)
  view.addModelsAsFrames(system)

  view.setStyle({'chain':'A'}, {'cartoon':{'color':'blue'}})
  view.setStyle({'chain':'B'}, {'cartoon':{'color':'yellow'}})
  view.zoomTo()
  view.show()


  display(pd.DataFrame(selected_subject))


#button.on_click(on_button_clicked)



## Display table 

In [29]:
data2 = data.style.format({'subject': hyperlink, 'sf': hyperlink})
data2

,query,subject,lovo_finalScore,lovo_coverage,lovo_rmsd,lovo_gaps,lovo_relCov,lovo_relGaps,lovo_finalScoreNorm,tm_AliLen,tm_RMSD,tm_n_ident/n_aln,tm_TM-score (chain 2),tm_d0 (chain 2),tm_cov,fatcat_subject-len,fatcat_Twists,fatcat_ini-len,fatcat_ini-rmsd,fatcat_opt-equ,fatcat_opt-rmsd,fatcat_chain-rmsd,fatcat_Score,fatcat_align-len,fatcat_Gaps,fatcat_rel_score,fatcat_rel_align,cl=46456,cl=48724,cl=51349,cl=53931,cl=56572,cl=56835,cl=56992,pred,pred_proba,cl,cf,sf
3,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d1guia_,0.266420,135,3.262520,16,0.871000,0.118500,0.654878,136,3.290000,0.140000,0.654250,4.640000,0.877400,155,0,80,3.800000,131,3.030000,3.800000,120.750000,194,63,0.264800,0.845200,0,1,0,0,0,0,0,1,0.777703,cl=48724,cf=49784,49785
1,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d1cx1a1,0.270919,141,3.505395,21,0.933800,0.148900,0.683576,140,3.350000,0.114000,0.682590,4.580000,0.927200,151,2,112,3.230000,134,3.190000,4.640000,106.510000,188,54,0.246600,0.887400,0,1,0,0,0,0,0,1,0.591836,cl=48724,cf=49784,49785
2,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d1gu3a_,0.254955,129,3.294609,18,0.908500,0.139500,0.684069,131,3.290000,0.099000,0.681570,4.430000,0.922500,142,0,96,3.310000,125,3.110000,3.310000,110.860000,175,50,0.271700,0.880300,0,1,0,0,0,0,0,1,0.527407,cl=48724,cf=49784,49785
12,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d4bj0a1,0.294247,145,3.147365,21,0.878800,0.144800,0.679444,146,3.160000,0.089000,0.678230,4.790000,0.884800,165,0,104,3.900000,139,3.180000,3.900000,131.010000,195,56,0.272900,0.842400,0,1,0,0,0,0,0,1,0.513047,cl=48724,cf=49784,49785
14,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d4leja1,0.186949,167,24.820392,13,1.000000,0.077800,0.426513,102,4.450000,0.108000,0.414720,4.820000,0.610800,167,3,96,6.200000,120,3.180000,16.420000,105.180000,221,101,0.219100,0.718600,0,1,0,0,0,0,0,0,0.407610,cl=48724,cf=51181,51182
11,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d3rf3a1,0.176324,117,10.754425,16,0.921300,0.136800,0.528972,99,3.760000,0.101000,0.501760,4.180000,0.779500,127,1,104,3.010000,112,3.260000,13.510000,224.420000,124,12,0.623400,0.881900,1,0,0,0,0,0,0,0,0.313741,cl=46456,cf=47161,47220
5,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d1nc7a1,0.148697,99,13.884749,12,0.916700,0.121200,0.524569,87,3.350000,0.138000,0.526840,3.820000,0.805600,108,1,56,3.310000,78,3.100000,4.410000,81.290000,117,39,0.260500,0.722200,0,1,0,0,0,0,0,0,0.290532,cl=48724,cf=89231,89232
4,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d1j2ha_,0.160505,110,11.048299,8,0.982100,0.072700,0.546005,79,2.670000,0.063000,0.533970,3.900000,0.705400,112,1,88,1.570000,99,2.200000,7.200000,247.820000,107,8,0.794300,0.883900,1,0,0,0,0,0,0,0,0.216126,cl=46456,cf=46965,89009
6,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d1nzea_,0.152333,111,16.598669,11,0.991100,0.099100,0.518204,84,3.490000,0.083000,0.509360,3.900000,0.750000,112,2,96,3.030000,102,1.800000,15.560000,231.770000,106,4,0.742900,0.910700,1,0,0,0,0,0,0,0,0.211696,cl=46456,cf=47161,101112
13,WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb,d4f01a2,0.157462,95,10.251438,10,0.989600,0.105300,0.624928,81,2.670000,0.086000,0.623000,3.570000,0.843800,96,1,88,3.110000,89,2.440000,7.350000,217.470000,110,21,0.823700,0.927100,1,0,0,0,0,0,0,0,0.193445,cl=46456,cf=46996,100934


In [30]:
data_table.disable_dataframe_formatter()

Description of the columns
- query: Query name
- subject: Subject name

Lovoalign
- lovo_finalScore: Final score;
- lovo_coverage: Alignment coverage;
- lovo_rmsd: RMSD;
- lovo_gaps: # of gaps;
- lovo_relCov: proportion of the coverage and subject length
- lovo_relGaps: propotion of gaps to coverage
- lovo_finalScoreNorm: Normalized score;

TM-align
- tm_AliLen: Alignment length;
- tm_RMSD: RMSD;
- tm_n_ident/n_aln: proportion of # identical atom and aligned length;
- tm_TM-score (chain 2): TM-score normalized by subject;
- tm_d0 (chain 2): scale factor used to calculate TM-score; 
- tm_cov: coverage of the alignment (subject)

FATCAT
- fatcat_subject-len: subject length;
- fatcat_Twists: # of twists;
- fatcat_ini-len: Initial alignment length;
- fatcat_ini-rmsd: Initial RMSD;
- fatcat_opt-equ: # of equivalent positions in the alignment;
- fatcat_opt-rmsd: RMSD of aligned Cα atoms of the input structures with structural rearragement;
- fatcat_chain-rmsd: RMSD of aligned Cα atoms of the input structures without structural rearragement;
- fatcat_Score: Alignment score
- fatcat_align-len: Alignment length 
- fatcat_Gaps: # of gaps in the alignment
- fatcat_rel_score: proportion of the alignment score and maximum score
- fatcat_rel_align: proportion of the # of aligned position with subject length

Prediction
- pred: prediction (0: not remote homolog; 1: remote homolog)
- pred_proba: prediction probability

SCOPe annotation
- cl: subject SCOPe class
- cf: subject SCOPe fold
- sf: subject SCOPe superfamily

## Display protein structure alignment

In [31]:
print("Choose a query")
display(query_picker)
print("Choose the Subject")
display(subject_picker)

Choose a query


Dropdown(description='query', options=('WP_0004663221_495e4_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000…

Choose the Subject


Dropdown(description='subject', options=('d1guia_', 'd1cx1a1', 'd1gu3a_', 'd4bj0a1', 'd4leja1', 'd3rf3a1', 'd1…

In [32]:
# Choose a query and a subject in the cell above and run this cell to display
# the structure alignement performed by FATCAT.
# chain in blue: Query
# chain in yellow: Subject
display_str()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

,query,subject,lovo_finalScore,lovo_coverage,lovo_rmsd,lovo_gaps,lovo_relCov,lovo_relGaps,lovo_finalScoreNorm,tm_AliLen,...,cl=51349,cl=53931,cl=56572,cl=56835,cl=56992,pred,pred_proba,cl,cf,sf
3,WP_0004663221_495e4_unrelaxed_rank_001_alphafo...,d1guia_,0.26642,135,3.26252,16,0.871,0.1185,0.654878,136,...,0,0,0,0,0,1,0.777703,cl=48724,cf=49784,sf=49785
